In [2]:
import pandas as pd
import json

In [3]:
def get_intersection(list1, list2):
    intersection = list(set(list1).intersection(list2))
    return intersection

def check_comparing_with_oracle(json_data: dict):
    res = 0
    for class_id in json_data:
        json_data_copy = json_data[class_id].copy()
        json_data_copy.pop("feature_values")
        json_data_copy.pop("cumple_condicion")
        if len(json_data_copy) > 0:
            discovered_features = json_data_copy
            feature_values = json_data[class_id]["feature_values"]
            relevant_features = 0
            for k, v in feature_values.items():
                if k[:7] == "or_cond":
                    aux = get_intersection(discovered_features, v)
                    relevant_features = relevant_features + 1 if len(aux) >= 1 else relevant_features
                else:
                    relevant_features = relevant_features + 1 if k in discovered_features else relevant_features
            rel_fe_in_tree = relevant_features / len(json_data_copy)
            if (res < rel_fe_in_tree):
                res = rel_fe_in_tree
    return res

In [9]:
real_or_mockups = "Real"
keys = {
    98: "IS_Rev1_" + real_or_mockups + "_FE1",
    99: "IS_Rev1_" + real_or_mockups + "_FE2",
    100: "IS_Rev1_" + real_or_mockups + "_FE3",
    101: "IS_Rev1_" + real_or_mockups + "_FE2+3",
    102: "IS_Rev1_" + real_or_mockups + "_FE1+2+3"
    # 76: "IS_Rev1_Real_FE1",
    # 77: "IS_Rev1_Real_FE2",
    # 78: "IS_Rev1_Real_FE3",
    # 79: "IS_Rev1_Real_FE2+3",
    # 80: "IS_Rev1_Real_FE1+2+3"
}

total_number_of_branches_in_dp = 4
# Crear una lista para almacenar los datos de cada experimento
experiment_data = []

for i in keys.keys():
    # Tu archivo JSON (reemplaza 'ruta_al_archivo.json' con la ruta real del archivo)
    id = str(i)
    json_file = "../../../resources/metadata/" + id + '-metainfo.json'

    # Leer el archivo JSON
    with open(json_file, 'r') as file:
        data = json.load(file)

    aux = str(keys[i]).split("_")
    # Procesar cada clave del JSON y extraer la información necesaria
    for experiment, info in data.items():
        feature_values = info['decision_tree_training']['feature_checker']["1"]["feature_values"]
            
        branch_correctly_classified = 0
        for class_id in info['decision_tree_training']['feature_checker']:
            fes = info['decision_tree_training']['feature_checker'][class_id]
            if fes["cumple_condicion"]:
                branch_correctly_classified += 1
        #     if len(fes) - 1 > relevant_fe:
        #         relevant_fe = len(fes)-1
        # features_discovered_in_tree = relevant_fe / len(feature_values)
        size = experiment.split("_")[2][4:]
        fes = aux[3].replace("FE", "")
        experiment_data.append([
            id,
            keys[i],
            aux[2][0]+"_"+size+"_"+fes,
            size,
            "_IMB" if "Imbalanced" in experiment else "_BAL",
            size+"_IMB" if "Imbalanced" in experiment else size+"_BAL",
            aux[2],
            "X" if "1" in str(aux[3]) else "",
            "X" if "2" in str(aux[3]) else "",
            "X" if "3" in str(aux[3]) else "",
            experiment,
            info['decision_tree_training']['sklearn']['duration'],
            info['decision_tree_training']['columns_len'],
            info['decision_tree_training']['accuracy']['accuracy'],
            info['decision_tree_training']['accuracy']['precision'],
            info['decision_tree_training']['accuracy']['recall'],
            info['decision_tree_training']['accuracy']['f1_score'],
            check_comparing_with_oracle(info['decision_tree_training']['feature_checker']),
            branch_correctly_classified / total_number_of_branches_in_dp
        ])

# Crear un DataFrame de pandas con los datos
columns = ['id', 'name', 'compact_name', 'logEvents', 'compact_balanced', 'logEvents_balanced','type', 'FE1', 'FE2', 'FE3', 'experiment', 'tree training duration', 'columns_len', 'accuracy', 'precision', 'recall', 'f1_score', 'features_discovered_in_tree', 'branch_correctly_classified']
df = pd.DataFrame(experiment_data, columns=columns)

# Guardar el DataFrame en un archivo CSV
output_csv = '../../../resources/resultados_experimentos_' + real_or_mockups + '.csv'
df.to_csv(output_csv, index=False)

print(f'Se ha creado el archivo CSV: {output_csv}')


Se ha creado el archivo CSV: ../../../resources/resultados_experimentos_Real.csv
